# Data Scientist

In [ ]:
from AriesExchanger import AriesDuetTokenExchanger

In [ ]:
%autoawait
import time
import asyncio
import nest_asyncio
from aries_basic_controller.aries_controller import AriesAgentController

    
# Anyone know how to get env variables into juypter notebooks through docker easily?
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://0.0.0.0:8021"

nest_asyncio.apply()

# Note AttachmentController just extends the basic controller to include the new attachment endpoint
scientist_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

In [ ]:
datascientist_exchanger = AriesDuetTokenExchanger(agent_controller=scientist_controller)

## Get Credential from Duet Issuer 

Again you should produce this invite through the relevant entrypoint exposed by the OM Authority user interface.

Accepting a connection using the correct invite from the duet issuer will result in the scientists agent being issued a credential from the Duet Issuer which they can use to authenticate with the Data Owner.

Note: The process by which the Data Scientist gets a credential can be designed to provide context specific trust to the use case. They might need to fill out a form, get approval from an ethics committee or prove they are a member of an organisation. The credential acts as a digital attestation to the successful completion of the defined process made by a trustworthy entity.

In [ ]:
invite = {"@type":"did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation","@id":"b0f83385-b5c7-4551-a5fb-b290c1a9d054","label":"OM_AUTHORITY","recipientKeys":["Ezh7Anu7uhP1Gqfcawj9b1vuw7VXiKZq9cHTutnbVzQV"],"serviceEndpoint":"https://0a0f5987e371.ngrok.io"}

In [ ]:
datascientist_exchanger.accept_connection(invite)

## Configure Duet Server Challenge

This is a aries proof request that is made to every entity who attempts to make a connection with the duet server. Before receiving the relevant token they must respond with a valid presentation to this request.

Note this challenge could be anything you want, designed to establish context specific trust dependant on the use case. For the challenge to be useful you must be confident that the correct entities will be able to get access to the credential needed to respond. 

In this example case they need a credential based on a schema with this ID `Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1`. Getting this credential is easy, entities simply need to connect to the Duet Issuer and will be automatically issued one. An example of a more realistic scenario could include application for research approval from an ethics committee.

In [ ]:
schema_id =  "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1"


revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
#TODO - change restriction to schemaId or credentialId

req_attrs = [
    {"name": "domain", "restrictions": [{"schema_id": schema_id}]},
]

global indy_proof_request
indy_proof_request = {
    "name": "Proof of Scientist",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
}

In [ ]:
datascientist_exchanger.configure_challenge(indy_proof_request)

## Join Duet Session

In this instance rather than launching a new duet server, we are going to join the one created by the Data Owner. Again we pass in a AriesExchanger as the DuetTokenExchanger. This time instead of creating an invitation, the duet instance requests a invitation be submitted as an input. 

Once this is done, the data scientists agent will establish a connection with the data owner and and authentication policies that were configured will be requested. If successful then the respective parties will communicate their duet token identifiers over the authenticated DIDComm channel. This will be used to establish a duet WebRTC connection across with PPML messages can be exchanged.

In [ ]:
import syft as sy
duet = sy.join_duet(credential_exchanger=datascientist_exchanger)

## Congratulations!

Once a duet session has been created, you should be able to run any PPML flows that duet/PySyft supports. This is left as an excercise for the interested. Do let me know if you run into any problems, but this has been designed so that the authentication layer is not tightly coupled to the Syft flows. Rather it act's as an entrypoint to a PPML flow.

In [ ]:
duet.torch